In [1]:
import statsapi
import pandas as pd
import requests
import sqlite3
from datetime import date, datetime

today = date.today().isoformat()

# --- Step 1: Fetch standings for both AL + NL ---
clean_rows = []
for league_id in [103, 104]:  # AL=103, NL=104
    standings_data = statsapi.get("standings", {"leagueId": league_id, "season": 2025, "date": today})
    for rec in standings_data.get("records", []):
        league = rec.get("league", {}) or {}
        division = rec.get("division", {}) or {}
        for tr in rec.get("teamRecords", []):
            team = tr.get("team", {}) or {}
            clean_rows.append({
                "season": 2025,
                "standings_date": today,
                "league_id": league.get("id"),
                "division_id": division.get("id"),
                "team_id": team.get("id"),
                "team_name": team.get("name"),
                "wins": tr.get("wins"),
                "losses": tr.get("losses"),
                "pct": tr.get("winningPercentage"),
                "games_back": tr.get("gamesBack"),
                "wc_games_back": tr.get("wildCardGamesBack"),
                "division_leader": tr.get("divisionLeader"),
                "created_at": datetime.now(),
                "last_updated": datetime.now()
            })

df_standings = pd.DataFrame(clean_rows)



In [2]:
# --- Step 2: Enrich with League + Division lookups ---
leagues = requests.get("https://statsapi.mlb.com/api/v1/leagues").json()["leagues"]
df_leagues = pd.DataFrame([{"league_id": l["id"], "league_name": l["name"]} for l in leagues])

divs = requests.get("https://statsapi.mlb.com/api/v1/divisions").json()["divisions"]
df_divs = pd.DataFrame([{"division_id": d["id"], "division_name": d["name"]} for d in divs])

df_standings = df_standings.merge(df_leagues, on="league_id", how="left")
df_standings = df_standings.merge(df_divs, on="division_id", how="left")

print("✅ Shape:", df_standings.shape)
print(df_standings.head(5))

✅ Shape: (30, 16)
   season standings_date  league_id  division_id  team_id          team_name  \
0    2025     2025-09-09        103          201      141  Toronto Blue Jays   
1    2025     2025-09-09        103          201      147   New York Yankees   
2    2025     2025-09-09        103          201      111     Boston Red Sox   
3    2025     2025-09-09        103          201      139     Tampa Bay Rays   
4    2025     2025-09-09        103          201      110  Baltimore Orioles   

   wins  losses   pct games_back wc_games_back  division_leader  \
0    82      61  .573          -             -             True   
1    80      63  .559        2.0          +4.5            False   
2    80      65  .552        3.0          +3.5            False   
3    71      72  .497       11.0           4.5            False   
4    66      77  .462       16.0           9.5            False   

                  created_at               last_updated      league_name  \
0 2025-09-09 11:06:01.

In [3]:
# --- Step 3: Write to SQLite ---
db_path = r"C:\Users\sneal\Dev\mlb-data-pipeline\mlb_data.db"
conn = sqlite3.connect(db_path)

df_standings.to_sql("standings_silver", conn, if_exists="replace", index=False)

conn.close()

print(f"✅ standings_silver written: {len(df_standings)} rows")


✅ standings_silver written: 30 rows
